In [ ]:
import os
import json
from tqdm import tqdm
from glob import glob
import numpy as np

import avstack
import avapi

path_to_results = "../../../../outputs/last_run"

In [ ]:
# Load in the metadata
with open(os.path.join(path_to_results, "launch_metadata.json"), 'r') as f:
    launch_metadata = json.load(f)
with open(os.path.join(path_to_results, "simulator", "metadata.json"), "r") as f:
    sim_metadata = json.load(f)

# Set up the carla dataset manager
CSM = avapi.carla.CarlaScenesManager(data_dir=sim_metadata["dataset_path"])
CDM = CSM.get_scene_dataset_by_index(sim_metadata["scene_idx"])
all_ts = np.array(CDM.timestamps)

# Get the frame/time mappings
with open(os.path.join(path_to_results, "simulator", "idx_to_frame_map.txt"), "r") as f:
    idx_to_frame_map = {int(line.rstrip().split()[0]): int(line.rstrip().split()[1]) for line in f.readlines()}
frame_to_idx_map = {v:k for k, v in idx_to_frame_map.items()}

# Load in the agent paths
output_paths = {}
frame_to_output_index = {}
for agent in ["ego"] + [f"agent{i}" for i in range(launch_metadata["n_infrastructure_agents"])]:
    output_paths[agent] = {}
    frame_to_output_index[agent] = {}
    for subfolder in ["detections", "tracks"]:
        path_to_sub = os.path.join(path_to_results, agent, subfolder)
        files_in_folder = [os.path.basename(file) for file in sorted(glob(os.path.join(path_to_sub, "*.txt")))]
        output_paths[agent][subfolder] = {}
        frame_to_output_index[agent][subfolder] = {}
        for i, file in enumerate(files_in_folder):
            ts = float(file.split('-')[3].replace('.txt', ''))
            frame = CDM.frames[np.argmin(abs(all_ts - ts))]
            idx = frame_to_idx_map[frame]
            output_paths[agent][subfolder][i] = {
                "timestamp":ts,
                "idx": idx,
                "frame": frame,
                "path": os.path.join(path_to_sub, file)
            }
            frame_to_output_index[agent][subfolder][frame] = i
        print("Recognized {} files for agent {} in subfolder {}".format(len(files_in_folder), agent, subfolder))

In [ ]:
def _get_base(agent, frame, subfolder, decoder):
    # load up the tracks
    index = frame_to_output_index[agent][subfolder][frame]
    file = output_paths[agent][subfolder][index]["path"]
    with open(file, "rb") as f:
        objects = json.load(f, cls=decoder)

    # replace passive reference with active
    for obj in objects:
        if isinstance(obj.reference, avstack.geometry.PassiveReferenceFrame):
            if obj.reference.frame_id == "ego":
                reference = CDM.get_ego_reference(frame)
            else:
                raise NotImplementedError
            obj.reference = reference
            obj.where_is_t = "bottom"  # HACK for tracks
            obj.q.reference = reference  # HACK for tracks

    return objects


def get_detections(agent, frame):
    return _get_base(agent, frame, "detections", avstack.modules.perception.detections.DetectionContainerDecoder)


def get_tracks(agent, frame):
    return _get_base(agent, frame, "tracks", avstack.modules.tracking.tracks.TrackContainerDecoder)

## Visualize ego information

In [ ]:
idx_frame = 10
frame = CDM.get_frames(sensor="main_camera")[idx_frame]
img = CDM.get_image(frame=idx_frame, sensor="main_camera")
tracks = get_tracks(agent="ego", frame=frame)
objects = CDM.get_objects(frame=idx_frame, sensor="main_camera")

# visualize snapshot
avapi.visualize.snapshot.show_image_with_boxes(img, tracks, inline=True)

In [ ]:
# make movie
frames = list(frame_to_output_index[agent]["tracks"].keys())[:-2]
print("loading frames")
all_imgs = [CDM.get_image(frame=frame, sensor="main_camera") for frame in tqdm(frames)]
print("loading tracks")
all_tracks = [get_tracks(agent="ego", frame=frame) for frame in tqdm(frames)]
avapi.visualize.movie.make_movie(
    raw_imgs=all_imgs, boxes=all_tracks, save=True, name="ego_playback.mov", show_in_notebook=True
)